In [ ]:
#!pip install -U unetseg

In [ ]:
#!pip install tensorflow-estimator==2.7.0
#!pip install opencv-python==4.5.4.60

In [ ]:
#!pip install git+https://github.com/dymaxionlabs/satproc.git@dab1cdbef0ef3a83c9e100952d9632e17bb18619

# 1_ Entrenamiento

   El modelo de ML utiliza para entrenar un dataset compuesto por imágenes y máscaras para el entrenamiento. Estas ultimas son imagenes binarias que delimitan el objeto de interés, 1 donde esta y 0 donde no. 
   
## Generacion del data set de entrenamiento

   En esta etapa generamos el dataset de entrenamiento mediante el uso de **GDAL** y **Satproc**. Primero descargamos las imagenes satelitales y utilizamos GDAL para quedarnos con la combinacion de bandas deseada. Luego Utilizamos satproc para generar el dataset con todas las caracteristicas necesarias para el modelo: tamaño , cantidad de imagenes, rango de valores, etc.

Descargamos las imágenes del bucket

In [ ]:
#creamos la carpeta a donde descargaremos las imagenes
PATH_s1_images_train = "../images/"

!mkdir -p $PATH_s1_images_train

In [ ]:
# gdutil ls --> lista los archivos en un bucket
#!gsutil ls gs://

In [ ]:
# gdutil cp -r --> copia los archivos del bucket al directorio de destino
bucket = "gs://" 

!gsutil -m cp -r $bucket   $PATH_s1_images_train

Podemos verificar que los archivos fueron correctamente descargados

In [ ]:
#!ls $PATH_s1_images

Combinamos todo las imagenes resultantes en un solo vrt

In [ ]:
!gdalbuildvrt \
         ../images/file.vrt \
         ../images/*.tif

### Satproc

Con la herramienta **satproc_extract_chips** se generan, a partir de las imágenes descargadas del bucket, imágenes (chips) generalmente mas pequeñas y máscaras (utilizando las anotaciones de verdad de campo).

In [ ]:
size =120
step_size = 40

**OBS** : tanto las imagenes como el archivo vectorial deben tener la misma georeferencia, por ejemplo 4326

In [ ]:
path_to_files   = PATH_s1_images_train + "/.vrt" #carpeta a las imagenes
output_folder   = '../dataset/data_train/s1/2021/'+str(size)+'_'+str(step_size)+'/' #carpeta de destino del dataset
vector_file     = "../data/shp/gt/anotcs.geojson" # archivo vectorial de verdad de campo
vector_file_aoi = "../data/shp/aoi.geojson" #archivo vectorial con las zonas de interes

Ejecutamos satproc

In [ ]:
!satproc_extract_chips \
                $path_to_files \
                -o  $output_folder \
                --size $size \
                --step-size $step_size \
                --aoi $vector_file_aoi \
                --labels $vector_file \
                --label-property 'class' \
                --classes 'A' \
                --rescale \
                --rescale-mode values --min -15 --max 5

###### Los argumentos:

* **path_to_files** es la ruta a las imágenes 

* **o** es la ruta de destino 

Recomendamos que dicha ruta sea descriptiva, por ejemplo “data_train/120_40/ ” describe : Data_train → datos usados para entrenar; 120_40 → <tamaño de la imagen >_ <tamaño del step-size> 

* **size** tamaño de las imágenes resultantes o chips (las imágenes son cuadradas) 
* **step-size** paso del proceso. Debe ser menor o igual a *size*. Si *step-size* es igual que el *size* entonces no hay overlap en las imágenes resultantes. 

En ocaciones es útil para el entrenamiento generar los chips con un overlap de este modo tenemos más datos para entrenar. 


* **label-property** nombre del campo donde se define cada categoría (solo se usa para el entrenamiento) 

* **classes** nombres de las clases (como aparecen en el geojson), separados por espacios

* **aoi** ruta al archivo vectorial donde están definidas las anotaciones. Al definir una region de interés solo se procesan las imágenes que interceptan esas anotaciones.

* **rescale** lleva los valores de las bandas a 0-255 

Este comando va a generar dos carpetas en la ruta de destino : “images” y “extent”. Los archivos de la primera van a ser de tipo Tiff de 3 bandas (rgb) y los de la segunda van a ser, también, de tipo Tiff pero de N bandas donde N representa el número de clases, en este caso sólo una. Y donde cada una de las bandas es una máscara binaria


## Entrenamiento

Generamos el entrenamiento del modelo utilizando los datasets creados en el paso previo. El modelo es una red neuronal CNN basado en la arquitectura U-Net. Este considera las imágenes y las máscaras binarias como inputs y genera una imagen con la probabilidad de encontrar al objeto de interés.

In [ ]:
from unetseg.train import TrainConfig, train
from unetseg.evaluate import plot_data_generator
import os

En esta etapa debemos definir la configuración del modelo de ML.

In [ ]:
config = TrainConfig(width=160,  #  tamaño de la imagen procesada por la UNet (debe ser multiplos de 16 , por ej 160, 320,etc; y no menor a 80)
                     height=160,
                     n_channels=3,  #  número de canales de la imagen, rgb -> 3
                     n_classes=1, # número de clases a clasificar
                     apply_image_augmentation=True, #  si es True , amplia el dataset generando imagenes nuevas a partir de pequeñas variaciones de las ya existentes (rotación,)
                     seed=42,
                     epochs=20, # Cantidad de veces que el dataset entero puede pasar por el proceso de entrenamiento
                     batch_size=16, #cantidad de datos que se procesan por vez, puede ser limitado por la memoria de gpu disponible (debe ser multiplo de 16)
                     steps_per_epoch=600, #  típicamente debe ser igual al numero de imágenes / el batch_size, si es mayor incrementara el número de imágenes generadas con image augmentation
                     early_stopping_patience=5, # a medida que entrena se guardan los resultados del entrenamiento despues de cada epoch, si el error no varió luego de ¿¿10 ?? iteraciones , se corta el proceso porque se entiende que el error ya disminuyó significativamente 
                     validation_split=0.2, # se divide la muestra en training (80% de los datos) y validation (20% de los datos) para calcular el error durante el proceso de entrenamiento
                     test_split=0.1, # Cantidad de imágenes del dataset
                     images_path=os.path.join('../dataset/data_train/s1','2021',str(size)+"_"+str(step_size)),#ruta a las imágenes generadas con Satproc
                     model_path=os.path.join('../data/weights/', 'UNet_160x160_spe100_3N_spe600_.h5'),#  ruta del modelo entrenado
                     model_architecture='unet',
                     evaluate=True  ,
                     class_weights= [1]) 

*Obs*: Es util usar un nombre para el archivo de pesos que de información sobre los parametros de entrenamiento. por ejemplo: < modelo >_< proyecto >_< dim_de_las_imagenes >_< size >_< step_size >_< step_per_epoch >.h5 o similares

Podemos visualizar alguna de las imágenes y máscaras del dataset de entrenamiento. A la izquierda se muestra la imágen y a la derecha la máscara 

In [ ]:
plot_data_generator(num_samples=3, fig_size=(10, 10), train_config=config,img_ch = 3)

Corremos el entrenamiento.

In [ ]:
res_config = train(config)

Graficamos las métricas generadas.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(16,4))

plt.subplot(121)
plt.plot(res_config.history['loss'])
plt.plot(res_config.history['val_loss'])
plt.title('Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')

plt.subplot(122)
plt.plot(res_config.history['mean_iou'])
plt.plot(res_config.history['val_mean_iou'])
plt.title('mean_iou')
plt.ylabel('val_mean_iou')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')


plt.show()
